In [1]:
import ee
import geemap
from geemap import ml
from sklearn import ensemble
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


# Inicializa la autenticación y la inicialización de Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-facuboladerasgee')

In [22]:
df1 = pd.read_csv("Datos_RF_0.csv")
df2 = pd.read_csv("Datos_RF_2.csv")
df3 = pd.read_csv("Datos_RF_3.csv")
df4 = pd.read_csv("Datos_RF_4.csv")
df5 = pd.read_csv("Datos_RF_5.csv")
df6 = pd.read_csv("Datos_RF_6.csv")
df7 = pd.read_csv("Datos_RF_7.csv")
df8 = pd.read_csv("Datos_RF_8.csv")
df9 = pd.read_csv("Datos_RF_9.csv")

# Unir todos los DataFrames en uno solo
df_combined = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9], ignore_index=True)

df_combined

,system:index,B11,B12,B2,B3,B4,B5,B6,B7,B8,B8A,CH,MNDWI,NDVI,VH,VV,agbd,agbd_t_se,label,.geo
0,0,2550.3333,1758.00,555.5000,799.0000,1036.0000,1435.0000,1962.3334,2165.00,2416.0,2517.0000,0.0,-0.522890,0.399768,-20.484844,-13.187520,1.500722,1.639169,4.0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
1,1,2607.0000,1741.25,512.3333,779.6667,1001.3333,1397.3334,1986.5000,2223.50,2434.0,2559.0000,0.0,-0.539567,0.417039,-20.466957,-13.368956,1.500722,1.639169,4.0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
2,2,2555.3333,1722.00,488.0000,748.5000,968.0000,1350.7500,1981.0000,2313.00,2492.0,2543.0000,0.0,-0.546890,0.440462,-20.732477,-13.241324,1.500722,1.639169,4.0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
3,3,2584.0000,1695.00,527.7500,768.7500,996.3333,1424.2500,1964.0000,2232.75,2415.0,2551.3333,0.0,-0.541421,0.415869,-20.898123,-13.395385,1.500722,1.639169,4.0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
4,4,2613.0000,1830.00,491.0000,738.6667,900.3333,1335.0000,1962.3334,2211.50,2312.0,2499.0000,0.0,-0.559224,0.439452,-20.074118,-13.653182,1.171597,1.639680,4.0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13929,1369,3018.0000,2216.50,800.0000,1098.0000,1299.0000,1468.5000,2142.5000,2312.00,2668.0,2565.5000,0.0,-0.466472,0.345097,-19.430996,-12.602765,1.214101,1.639609,4.0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
13930,1370,3033.0000,2153.50,553.5000,846.5000,946.0000,1434.5000,2100.5000,2468.00,2503.0,2693.5000,0.0,-0.563604,0.451435,-19.399767,-12.168163,1.214101,1.639609,4.0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
13931,1371,2569.0000,1911.50,436.5000,645.5000,782.0000,1059.0000,1858.0000,2196.50,2362.0,2423.5000,0.0,-0.598382,0.502544,-19.388850,-12.649703,1.271942,1.639515,4.0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
13932,1372,2625.5000,1774.50,445.5000,675.0000,757.0000,1059.5000,1696.0000,1972.00,2079.0,2252.5000,0.0,-0.590971,0.466149,-20.414808,-12.551848,1.271942,1.639515,4.0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."


In [26]:
# Calcular la nueva columna 'threshold'
df_combined['threshold'] = (df_combined['agbd_t_se'] / df_combined['agbd']) * 100

df_combined

,system:index,B11,B12,B2,B3,B4,B5,B6,B7,B8,...,CH,MNDWI,NDVI,VH,VV,agbd,agbd_t_se,label,.geo,threshold
0,0,2550.3333,1758.00,555.5000,799.0000,1036.0000,1435.0000,1962.3334,2165.00,2416.0,...,0.0,-0.522890,0.399768,-20.484844,-13.187520,1.500722,1.639169,4.0,"{""geodesic"":false,""type"":""Point"",""coordinates""...",109.225360
1,1,2607.0000,1741.25,512.3333,779.6667,1001.3333,1397.3334,1986.5000,2223.50,2434.0,...,0.0,-0.539567,0.417039,-20.466957,-13.368956,1.500722,1.639169,4.0,"{""geodesic"":false,""type"":""Point"",""coordinates""...",109.225360
2,2,2555.3333,1722.00,488.0000,748.5000,968.0000,1350.7500,1981.0000,2313.00,2492.0,...,0.0,-0.546890,0.440462,-20.732477,-13.241324,1.500722,1.639169,4.0,"{""geodesic"":false,""type"":""Point"",""coordinates""...",109.225360
3,3,2584.0000,1695.00,527.7500,768.7500,996.3333,1424.2500,1964.0000,2232.75,2415.0,...,0.0,-0.541421,0.415869,-20.898123,-13.395385,1.500722,1.639169,4.0,"{""geodesic"":false,""type"":""Point"",""coordinates""...",109.225360
4,4,2613.0000,1830.00,491.0000,738.6667,900.3333,1335.0000,1962.3334,2211.50,2312.0,...,0.0,-0.559224,0.439452,-20.074118,-13.653182,1.171597,1.639680,4.0,"{""geodesic"":false,""type"":""Point"",""coordinates""...",139.952560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13929,1369,3018.0000,2216.50,800.0000,1098.0000,1299.0000,1468.5000,2142.5000,2312.00,2668.0,...,0.0,-0.466472,0.345097,-19.430996,-12.602765,1.214101,1.639609,4.0,"{""geodesic"":false,""type"":""Point"",""coordinates""...",135.047167
13930,1370,3033.0000,2153.50,553.5000,846.5000,946.0000,1434.5000,2100.5000,2468.00,2503.0,...,0.0,-0.563604,0.451435,-19.399767,-12.168163,1.214101,1.639609,4.0,"{""geodesic"":false,""type"":""Point"",""coordinates""...",135.047167
13931,1371,2569.0000,1911.50,436.5000,645.5000,782.0000,1059.0000,1858.0000,2196.50,2362.0,...,0.0,-0.598382,0.502544,-19.388850,-12.649703,1.271942,1.639515,4.0,"{""geodesic"":false,""type"":""Point"",""coordinates""...",128.898566
13932,1372,2625.5000,1774.50,445.5000,675.0000,757.0000,1059.5000,1696.0000,1972.00,2079.0,...,0.0,-0.590971,0.466149,-20.414808,-12.551848,1.271942,1.639515,4.0,"{""geodesic"":false,""type"":""Point"",""coordinates""...",128.898566


In [28]:
# Filtrar los datos cuyo valor en la columna 'threshold' sea menor que 50
df_filtered = df_combined[df_combined['threshold'] < 50]

# Mostrar las primeras filas del DataFrame filtrado para verificar
df_filtered

,system:index,B11,B12,B2,B3,B4,B5,B6,B7,B8,...,CH,MNDWI,NDVI,VH,VV,agbd,agbd_t_se,label,.geo,threshold
148,148,2301.0000,1387.5,397.00000,628.00000,630.5,1191.0,2544.5,3003.5000,3181.0,...,3.0,-0.571185,0.669159,-19.448360,-12.503587,11.029232,1.633992,4.0,"{""geodesic"":false,""type"":""Point"",""coordinates""...",14.815102
236,236,2175.5000,1224.0,383.50000,624.33330,591.0,1113.5,2138.0,2562.5000,2720.0,...,4.0,-0.554021,0.643008,-18.295868,-12.306431,4.105282,1.636674,2.0,"{""geodesic"":false,""type"":""Point"",""coordinates""...",39.867517
237,237,2286.5000,1350.0,456.00000,719.00000,722.0,1257.5,2455.0,2876.0000,3134.0,...,4.0,-0.521544,0.625519,-19.432308,-13.394172,4.105282,1.636674,4.0,"{""geodesic"":false,""type"":""Point"",""coordinates""...",39.867517
375,375,1483.3334,773.0,208.33333,425.00000,272.0,763.0,2162.5,2654.0000,2963.0,...,8.0,-0.554585,0.831839,-15.522433,-9.955149,46.720367,1.635166,1.0,"{""geodesic"":false,""type"":""Point"",""coordinates""...",3.499900
376,376,1562.0000,797.5,239.00000,472.33334,322.0,802.0,2346.0,2887.6667,3152.0,...,8.0,-0.535638,0.814623,-13.531593,-7.769205,46.720367,1.635166,1.0,"{""geodesic"":false,""type"":""Point"",""coordinates""...",3.499900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13845,1285,1940.0000,1143.0,322.00000,488.00000,409.0,915.0,2209.5,2602.5000,2540.0,...,10.0,-0.598023,0.722618,-16.868895,-10.457448,4.289997,1.636552,1.0,"{""geodesic"":false,""type"":""Point"",""coordinates""...",38.148092
13848,1288,2383.0000,1421.0,303.00000,558.00000,436.0,1134.0,2410.0,2823.0000,3056.0,...,10.0,-0.620537,0.750286,-16.140898,-8.957723,4.289997,1.636552,1.0,"{""geodesic"":false,""type"":""Point"",""coordinates""...",38.148092
13882,1322,1598.5000,857.5,182.00000,367.00000,245.0,697.5,2393.0,2820.0000,3168.0,...,15.0,-0.626558,0.856431,-14.653900,-8.987279,11.980653,1.633802,1.0,"{""geodesic"":false,""type"":""Point"",""coordinates""...",13.637003
13908,1348,2195.0000,1295.0,289.00000,544.00000,417.0,984.0,2061.0,2406.0000,2554.0,...,6.0,-0.602775,0.719286,-16.975485,-10.473569,17.841530,1.633113,1.0,"{""geodesic"":false,""type"":""Point"",""coordinates""...",9.153436


In [29]:
# Obtener los nombres de las columnas excepto 'agbd' y '.geo'
features = [col for col in df_filtered.columns if col not in ['agbd', '.geo', 'system:index']]

# Mostrar la lista de características
print(features)

['B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'CH', 'MNDWI', 'NDVI', 'VH', 'VV', 'agbd_t_se', 'label', 'threshold']


In [36]:
# Separar la variable objetivo 'agbd'
y = df_filtered['agbd']

# Separar las variables independientes (todas excepto 'agbd' y '.geo')
X = df_filtered.drop(columns=['agbd','agbd_t_se', 'threshold','.geo', 'system:index'])

X

,B11,B12,B2,B3,B4,B5,B6,B7,B8,B8A,CH,MNDWI,NDVI,VH,VV,label
148,2301.0000,1387.5,397.00000,628.00000,630.5,1191.0,2544.5,3003.5000,3181.0,3307.5000,3.0,-0.571185,0.669159,-19.448360,-12.503587,4.0
236,2175.5000,1224.0,383.50000,624.33330,591.0,1113.5,2138.0,2562.5000,2720.0,2873.1667,4.0,-0.554021,0.643008,-18.295868,-12.306431,2.0
237,2286.5000,1350.0,456.00000,719.00000,722.0,1257.5,2455.0,2876.0000,3134.0,3192.5000,4.0,-0.521544,0.625519,-19.432308,-13.394172,4.0
375,1483.3334,773.0,208.33333,425.00000,272.0,763.0,2162.5,2654.0000,2963.0,2932.0000,8.0,-0.554585,0.831839,-15.522433,-9.955149,1.0
376,1562.0000,797.5,239.00000,472.33334,322.0,802.0,2346.0,2887.6667,3152.0,3198.0000,8.0,-0.535638,0.814623,-13.531593,-7.769205,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13845,1940.0000,1143.0,322.00000,488.00000,409.0,915.0,2209.5,2602.5000,2540.0,2904.0000,10.0,-0.598023,0.722618,-16.868895,-10.457448,1.0
13848,2383.0000,1421.0,303.00000,558.00000,436.0,1134.0,2410.0,2823.0000,3056.0,3061.0000,10.0,-0.620537,0.750286,-16.140898,-8.957723,1.0
13882,1598.5000,857.5,182.00000,367.00000,245.0,697.5,2393.0,2820.0000,3168.0,3140.0000,15.0,-0.626558,0.856431,-14.653900,-8.987279,1.0
13908,2195.0000,1295.0,289.00000,544.00000,417.0,984.0,2061.0,2406.0000,2554.0,2652.0000,6.0,-0.602775,0.719286,-16.975485,-10.473569,1.0


In [37]:
feature_names = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12', 'NDVI', 'MNDWI', 'VV', 'VH', 'label', 'CH']
label = "agbd"


In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [39]:
rf = RandomForestRegressor(random_state=42)

# Definir el grid de parámetros para buscar
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20 ],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Configurar GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Entrenar GridSearchCV
grid_search.fit(X_train, y_train)

# Obtener los mejores parámetros
best_params = grid_search.best_params_
print(f'Best parameters: {best_params}')

# Entrenar el modelo con los mejores parámetros
best_rf = grid_search.best_estimator_
best_rf.fit(X_train, y_train)

# Hacer predicciones
y_pred = best_rf.predict(X_test)

# Calcular las métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

Fitting 3 folds for each of 162 candidates, totalling 486 fits
Best parameters: {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Mean Squared Error: 941.3747541276812
R^2 Score: 0.5446661201860342


In [40]:
# convert the estimator into a list of strings
# this function also works with the ensemble.ExtraTrees estimator
trees = ml.rf_to_strings(best_rf, feature_names,output_mode='regression')
# print the first tree to see the result
# print(trees[0])

In [ ]:
# create a ee classifier to use with ee objects from the trees
ee_classifier = ml.strings_to_classifier(trees)
# ee_classifier.getInfo()

In [ ]:
# classify the image using the classifier we created from the local training
# note: here we select the feature_names from the image that way the classifier knows which bands to use
classified = image.select(feature_names).classify(ee_classifier)

In [41]:
user_id = 'users/facuboladerasgee'
# specify asset id where to save trees
# be sure to change  to your ee user name
asset_id = user_id + "/Rf_Gee"
asset_id

'users/facuboladerasgee/Rf_Gee'

In [42]:
# kick off an export process so it will be saved to the ee asset
ml.export_trees_to_fc(trees, asset_id)

# this will kick off an export task, so wait a few minutes before moving on

In [ ]:
# read the exported tree feature collection
rf_fc = ee.FeatureCollection(asset_id)

# convert it to a classifier, very similar to the `ml.trees_to_classifier` function
another_classifier = ml.fc_to_classifier(rf_fc)

# classify the image again but with the classifier from the persisted trees
classified = image.select(feature_names).classify(another_classifier)